## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-22-16-52-46 +0000,nypost,Trump admin scraps 5 offshore wind farms— incl...,https://nypost.com/2025/12/22/us-news/trump-ad...
1,2025-12-22-16-50-44 +0000,nyt,"Christmas Is Back in Bethlehem, but Peace and ...",https://www.nytimes.com/2025/12/22/world/middl...
2,2025-12-22-16-47-49 +0000,nyt,Waymo Suspended Service in San Francisco After...,https://www.nytimes.com/2025/12/21/us/waymo-su...
3,2025-12-22-16-46-03 +0000,nyt,Ex-C.I.A. Chief Wants to Block Judge Cannon Fr...,https://www.nytimes.com/2025/12/22/us/politics...
4,2025-12-22-16-41-13 +0000,nyt,How U.S. Defense Industry Dodged a Rare-Earth ...,https://www.nytimes.com/2025/12/22/business/de...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2366/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,28
457,up,10
75,was,10
7,new,9
159,files,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
216,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...,60
231,2025-12-21-18-40-44 +0000,nyt,Epstein Files Photos Disappear From Government...,https://www.nytimes.com/2025/12/20/us/politics...,58
20,2025-12-22-15-49-12 +0000,bbc,New Trump envoy says he will serve to make Gre...,https://www.bbc.com/news/articles/ckgmd132ge4o...,57
227,2025-12-21-19-18-08 +0000,nypost,Nicki Minaj makes surprise appearance with Eri...,https://nypost.com/2025/12/21/us-news/nicki-mi...,57
41,2025-12-22-14-51-10 +0000,wapo,‘60 Minutes’ correspondent says CBS’s Bari Wei...,https://www.washingtonpost.com/business/2025/1...,55


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
216,60,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...
231,58,2025-12-21-18-40-44 +0000,nyt,Epstein Files Photos Disappear From Government...,https://www.nytimes.com/2025/12/20/us/politics...
23,54,2025-12-22-15-39-36 +0000,wapo,Car bomb in Moscow kills senior Russian genera...,https://www.washingtonpost.com/world/2025/12/2...
44,43,2025-12-22-14-47-26 +0000,latimes,Larry Ellison pledges $40 billion personal gua...,https://www.latimes.com/entertainment-arts/bus...
151,43,2025-12-22-06-38-22 +0000,nypost,Bondi Beach terrorists railed against ‘Zionist...,https://nypost.com/2025/12/22/world-news/polic...
241,40,2025-12-21-17-30-34 +0000,nypost,US military targeting third tanker off coast o...,https://nypost.com/2025/12/21/world-news/us-mi...
236,36,2025-12-21-17-57-50 +0000,latimes,'Avatar: Fire and Ash' heats up the box office...,https://www.latimes.com/entertainment-arts/bus...
221,33,2025-12-21-19-56-00 +0000,wsj,A top Justice Department official said Sunday ...,https://www.wsj.com/politics/policy/justice-de...
154,32,2025-12-22-06-01-44 +0000,latimes,CBS News correspondent accuses Bari Weiss of '...,https://www.latimes.com/entertainment-arts/bus...
2,26,2025-12-22-16-47-49 +0000,nyt,Waymo Suspended Service in San Francisco After...,https://www.nytimes.com/2025/12/21/us/waymo-su...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
